In [14]:
import sys
sys.path.append('/content/drive/MyDrive/ColabNotebooks/nyanko_MLP')

In [16]:
import torch
import numpy
from features.FeatureFunction2 import (
    AttackFeature_solo,AttackFeature_mass,
    DefenseFeature_solo,DefenseFeature_mass,
    DisturbFeature_solo,DisturbFeature_mass,Wave_DisturbFeature,
    CostFeature,AttributeFeature,
    WPOFeature,RoleFeature2,
    RangeFeature,HandleFeature,
    GimmickFeature,AtFreFeature
)
from features.FeatureFunctionSpirit2 import(
    AttackFeatureSpirit,
    DefenseFeatureSpirit,
    DisturbFeatureSpirit,
    WPOFeatureSpirit,
)
from model.Spirit_router import SpiritRouter
from normalizer.base import StandardScaler
from model.mlp import MLP,FullModel
from model.feature_concat import FeatureConcat
from data.Rawdata import Rawdata
from model.normalizedFM import NormalizedFeatureModel
from data.SaveManagerCosine import SaveManager
from data.NyankoDataset import NyankoDataset
from data.FeatureEncorder import FeatureEncorder,TensorEncorder

# ======================
# device
# ======================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

# ======================
# feature 定義（学習時と完全一致）
# ======================
features_normal = [
    AttackFeature_solo(device),
    AttackFeature_mass(device),
    DefenseFeature_solo(device),
    DefenseFeature_mass(device),
    DisturbFeature_solo(device),
    DisturbFeature_mass(device),
    Wave_DisturbFeature(device),
    CostFeature(device),
    AttributeFeature(device),
    WPOFeature(device),
    RoleFeature2(device),
    RangeFeature(device),
    HandleFeature(device),
    GimmickFeature(device),
    AtFreFeature(device)
]

features_spirit = [
    AttackFeature_solo(device),
    AttackFeatureSpirit(device),
    DefenseFeature_solo(device),
    DefenseFeatureSpirit(device),
    DisturbFeature_solo(device),
    DisturbFeatureSpirit(device),
    Wave_DisturbFeature(device),
    CostFeature(device),
    AttributeFeature(device),
    WPOFeatureSpirit(device),
    RoleFeature2(device),
    RangeFeature(device),
    HandleFeature(device),
    GimmickFeature(device),
    AtFreFeature(device)
]


# ======================
# Dataset（評価用 Excel）
# ※ mu, sigma は一切使わない
# ======================

rawdata = Rawdata("/content/drive/MyDrive/ColabNotebooks/nyanko_MLP/data/nyanko_DB100_dummy.xlsx")
'''
rawdata = Rawdata("/content/drive/MyDrive/ColabNotebooks/nyanko_MLP/gatya_data/kerihime_CC.xlsx")
'''
feature_encoder = FeatureEncorder()
tensor_encoder = TensorEncorder(device=device)  # CPU tensorのまま
dataset = NyankoDataset(rawdata, feature_encoder, tensor_encoder, target_col="評価値")

router = SpiritRouter(features_normal,features_spirit)

feature_concat = FeatureConcat(router)
# ======================
# ダミー mu, sigma を作る
# （shape が合っていれば値は何でもいい）
# ======================
sample_row = dataset.rows[0]

with torch.no_grad():
    dummy_feat = feature_concat(sample_row)

dummy_mu = torch.zeros_like(dummy_feat).to(device)
dummy_sigma = torch.ones_like(dummy_feat).to(device)

normalizer = StandardScaler(dummy_mu, dummy_sigma).to(device)

# ======================
# model 構築
# ======================
norm_feature_model = NormalizedFeatureModel(feature_concat, normalizer).to(device)

mlp = MLP(
    input_dim=dummy_feat.shape[0],
    hidden_dim=28
).to(device)

model = FullModel(norm_feature_model, mlp).to(device)

# ======================
# 学習済みモデルをロード
# （ここで mu, sigma が復元される）
# ======================
save_manager = SaveManager(
    "/content/drive/MyDrive/ColabNotebooks/nyanko_MLP/weight_data"
)


# saver は SaveManager のインスタンス
ckpt_state = save_manager.load_from_path(
    "/content/drive/MyDrive/ColabNotebooks/nyanko_MLP/weight_data/best.ckpt",
    model
)

# extra_state や epoch も確認できる
if "extra_state" in ckpt_state:
    print(ckpt_state["extra_state"])
print(ckpt_state["epoch"], ckpt_state["metric"])





feature_mu = ckpt_state["extra_state"]["feature_mu"].to(dtype=torch.float32,device=device)
feature_sigma = ckpt_state["extra_state"]["feature_sigma"].to(dtype=torch.float32,device=device)

feature_sigma = torch.where(feature_sigma==0,torch.ones_like(feature_sigma),feature_sigma)

normalizer.mu.copy_(feature_mu)
normalizer.sigma.copy_(feature_sigma)

y_mu = ckpt_state["extra_state"]["y_mu"].to(dtype=torch.float32,device=device)
y_sigma = ckpt_state["extra_state"]["y_sigma"].to(dtype=torch.float32,device=device)

def inverse_z(y_z):
    return y_z * y_sigma + y_mu

model.eval()

# ======================
# 推論
# ======================
with torch.no_grad():
    for i in range(len(dataset)):
        row = dataset.rows[i]
        y = dataset.targets[i]

        y_pred_z = model(row)
        y_pred = inverse_z(y_pred_z).item()

        print(i, row.get("name"), f"{y_pred:.2f}", y.item())


device: cpu
{'y_mu': tensor(5.5322), 'y_sigma': tensor(2.4917), 'feature_mu': tensor([1.5121e+01, 1.8147e+01, 1.9425e+02, 6.2918e+01, 8.1652e+00, 4.7051e+00,
        2.3915e+00, 1.7266e-01, 2.5085e+00, 2.8383e-01, 1.5254e-01, 1.2712e-01,
        2.1186e-01, 8.4030e+00, 3.1992e+01, 1.0169e-01, 0.0000e+00, 3.3898e-02,
        8.4746e-03, 0.0000e+00, 1.6949e-02, 2.1186e-01, 1.0169e-01, 6.7797e-02,
        1.0169e-01, 1.0169e-01, 1.1864e-01, 1.1864e-01, 8.4746e-03, 8.4746e-02,
        8.4746e-02, 3.3898e-02, 8.4746e-03, 3.3898e-02, 3.3898e-02, 5.9322e-02,
        5.9322e-02, 1.0136e+00]), 'feature_sigma': tensor([1.1628e+01, 1.1564e+01, 2.6763e+02, 4.3176e+01, 1.1655e+01, 1.0153e+01,
        2.2858e+01, 1.2374e-01, 3.2601e+00, 5.0648e-02, 3.6108e-01, 3.3453e-01,
        4.1037e-01, 8.5223e-01, 3.3987e+01, 3.0354e-01, 0.0000e+00, 1.8174e-01,
        9.2057e-02, 0.0000e+00, 1.2963e-01, 4.1037e-01, 3.0354e-01, 2.5247e-01,
        3.0354e-01, 3.0354e-01, 3.2475e-01, 3.2475e-01, 9.2057e-02, 2.7